In [1]:
import sys
import os

# Append the directory of the module to the Python path
sys.path.append("/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit")

In [2]:
from cell import Cell
from link_composer import LinkComposer
import cells_extractor 
import visualizer
import link_algorithm
from cell_event import CellEvent, CellDefine, Cell

In [3]:
mask_folder = "/Users/sherryyang/Documents/wiggins-lab/data/1119/xy5/masks/*.png"
phase_folder = "/Users/sherryyang/Documents/wiggins-lab/data/1119/xy5/phase/*c1.tif"
composer = LinkComposer.read_folder(mask_folder=mask_folder, phase_folder=phase_folder)

In [4]:
mask_tif = "/Users/sherryyang/Documents/wiggins-lab/kevins-data/teresa_high_frame_rate/12_edited_labels.tif"
composer = LinkComposer.read_tif(mask_tif=mask_tif)

In [5]:
mask_folder = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/masks_W02/*png"
phase_folder = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W02/*tif"
composer = LinkComposer.read_folder(mask_folder=mask_folder, phase_folder=phase_folder)

In [6]:
spots_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_spots_NN.csv"
edge_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_edges_NN.csv"
G_NN = composer.get_trackmate_linking_result(spots_filename, edge_filename, UNIT_CONVERT_COEFF= 1)

In [7]:
spots_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_spots_over.csv"
edge_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_edges_over.csv"
G_over = composer.get_trackmate_linking_result(spots_filename, edge_filename, UNIT_CONVERT_COEFF= 1)

In [8]:
spots_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_spots.csv"
edge_filename = "/Users/sherryyang/Documents/wiggins-lab/ImageProcessing-Benchmarking-main/raw-images/E.coli/workabla-data-set/W2_edges.csv"
G_lap = composer.get_trackmate_linking_result(spots_filename, edge_filename, UNIT_CONVERT_COEFF= 1)

In [9]:
import networkx as nx
from queue import Queue

In [10]:
composer.cells

{Cell: frame_0, label_1, <-0x1b119000fefce840>,
 Cell: frame_0, label_2, <0x63fca45ae467b492>,
 Cell: frame_0, label_3, <-0x41b392db3247e415>,
 Cell: frame_1, label_1, <0x746be2e2370f37ea>,
 Cell: frame_1, label_2, <-0x31445453dfa060bd>,
 Cell: frame_1, label_3, <-0x13fea646764ff964>,
 Cell: frame_2, label_1, <0x5e4c03d9f61c5545>,
 Cell: frame_2, label_2, <0x1a6452f25980f217>,
 Cell: frame_3, label_1, <0x7983f36362147888>,
 Cell: frame_3, label_2, <0x359c427bc579155a>,
 Cell: frame_4, label_1, <0x236b5661b35cb5c>,
 Cell: frame_4, label_2, <-0x7ebb163e016597d2>,
 Cell: frame_5, label_1, <0x2feba1fad72db60d>,
 Cell: frame_5, label_2, <-0x510629a9456dad21>,
 Cell: frame_5, label_3, <0x9499f20a3e2ba38>,
 Cell: frame_6, label_1, <0x6fe1fd163d3a0ec9>,
 Cell: frame_6, label_2, <-0x35ce3a1fd97589de>,
 Cell: frame_6, label_3, <0x24818eaa0fdadd7b>,
 Cell: frame_6, label_4, <-0x5c703cfa0cc085b3>,
 Cell: frame_7, label_1, <-0x6933c73583b8d3dc>,
 Cell: frame_7, label_2, <0x52e487e2dfabc8f6>,
 Cell:

In [11]:
def cut_graph(G):
    for u, v in G.edges():
        G[u][v]['reliability'] = sys.maxsize

    for node in G.nodes():
        edges_with_weights = G.edges(node, data='weight')
        sorted_edges = sorted(edges_with_weights, key=lambda x: x[2])
        for i, (u, v, weight) in enumerate(sorted_edges):
            G[u][v]['reliability'] = min(i, G[u][v]['reliability'])

    edges_with_rel=  G.edges(data='reliability')
    sorted_edges = sorted(edges_with_rel, key=lambda x: x[2], reverse=True)
    index = 0

    while nx.is_connected(G):
        u,v, realiability_target = sorted_edges[index]
        realiability = realiability_target
        while realiability == realiability_target:
            index += 1
            G.remove_edge(u, v)
            u,v, realiability_target = sorted_edges[index]

    return G

In [12]:
def link_subtree(G):
    all_edges = set(G.edges(data='weight'))
    test_G = nx.Graph()
    edges = set()
    for u, v, weight in all_edges:
        if G.degree(u) == 1 or G.degree(v) == 1:
            test_G.add_edge(u,v, weight = 0)
        else:
            edges.add((u,v, weight))

    edges = list(edges)
    test_G.add_nodes_from(G.nodes())

    min_cost = sys.maxsize
    min_tree = nx.Graph()

    min_cost, min_tree = link_subtree_helper(0, edges, test_G, min_cost, min_tree)
    return min_cost, min_tree



def link_subtree_helper(index, edges, test_G, min_cost, min_tree):
    if index == len(edges):
        cost = sum(weight for _, _, weight in test_G.edges(data='weight'))
        cost += len(list(nx.isolates(test_G))) * 10
        if cost < min_cost:
            min_cost = cost
            min_tree = test_G.copy()
        return min_cost, min_tree

    u, v, weight = edges[index]

    min_cost, min_tree = link_subtree_helper(index + 1, edges, test_G, min_cost, min_tree)
    if (test_G.degree(u) <= 1) or (test_G.degree(v) <= 1):
        test_G.add_edge(u, v, weight=weight)
        min_cost, min_tree = link_subtree_helper(index + 1, edges, test_G, min_cost, min_tree)
        test_G.remove_edge(u, v)

    return min_cost, min_tree

In [13]:
tree_set = set()

for frame in range(1,composer.frame_num):

    dict = composer.cells_frame_dict
    sorted_frame = sorted(dict)

    source_cells = list(dict[sorted_frame[frame-1]])
    target_cells =  list(dict[sorted_frame[frame]])

    source_cells.sort()
    target_cells.sort()

    source_dim = len(source_cells)
    target_dim = len(target_cells)

    sub_G = nx.Graph()
    for i in range(source_dim):
        for j in range(target_dim):
            source_cell = source_cells[i]
            target_cell = target_cells[j]

            intersection_area = source_cell.polygon.intersection(target_cell.polygon).area
            if intersection_area > 0:
                union_area = source_cell.polygon.union(target_cell.polygon).area
                weight = -1 * intersection_area /  union_area  + 1
                sub_G.add_edge(source_cell, target_cell, weight=weight)

    
    queue = Queue()
    queue.put(sub_G)

    while not queue.empty():
        sub_G = queue.get()
        components = nx.connected_components(sub_G)
        subgraphs = []
        for nodes in components:
            subgraph = sub_G.subgraph(nodes).copy()
            if len(subgraph.edges()) < 20:
                min_cost, min_tree = link_subtree(subgraph)
                tree_set.add(min_tree)
            else:
                subgraph = cut_graph(subgraph)
                queue.put(subgraph)

In [14]:
from shapely.ops import nearest_points

tree_set = set()

for frame in range(1,composer.frame_num):

    dict = composer.cells_frame_dict
    sorted_frame = sorted(dict)

    source_cells = list(dict[sorted_frame[frame-1]])
    target_cells =  list(dict[sorted_frame[frame]])

    source_cells.sort()
    target_cells.sort()

    source_dim = len(source_cells)
    target_dim = len(target_cells)

    sub_G = nx.Graph()
    for i in range(source_dim):
        for j in range(target_dim):
            source_cell = source_cells[i]
            target_cell = target_cells[j]

            intersection_area = source_cell.polygon.intersection(target_cell.polygon).area
            union_area = source_cell.polygon.union(target_cell.polygon).area
            IoU = -1 * intersection_area /  union_area  + 1

            distance = 0
            if intersection_area == 0:
                centroid_target = target_cell.polygon.centroid
                nearest_point = nearest_points(centroid_target , source_cell.polygon)[1]
                distance = centroid_target.distance(nearest_point)
                
            weight = distance + IoU
            sub_G.add_edge(source_cell, target_cell, weight=weight)

    
    queue = Queue()
    queue.put(sub_G)

    while not queue.empty():
        sub_G = queue.get()
        components = nx.connected_components(sub_G)
        subgraphs = []
        for nodes in components:
            subgraph = sub_G.subgraph(nodes).copy()
            if len(subgraph.edges()) < 20:
                min_cost, min_tree = link_subtree(subgraph)
                tree_set.add(min_tree)
            else:
                subgraph = cut_graph(subgraph)
                queue.put(subgraph)

In [15]:
G_new = composer.make_new_dircted_graph()
for graph in tree_set:
    for u, v in graph.edges():
        if u < v:
            G_new.add_edge(u,v)
        else:
            G_new.add_edge(v,u)

In [16]:
G = link_algorithm.silly_linker(composer)

In [17]:
"""
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from mpl_interactions import zoom_factory
from IPython.display import display
import matplotlib

plt.close('all')
matplotlib.use('TkAgg')  # Make sure this backend is compatible with your environment

frame = 0  # Initialize frame
max_frame = composer.frame_num - 1

# Assuming visualizer, composer, and G are defined
# Create the initial plot
plt.ioff()
fig, ax = plt.subplots(figsize=(10, 10))
fig.tight_layout()



label_info_1 = visualizer.get_label_info(G)
label_info_2 = visualizer.get_generation_label_info(G)


labels = [label_info_1, label_info_2]
li_index = 0
label_info = labels[li_index]


label_styles = ["regular", "circled", "empty"]
ls_index = 0
label_style = label_styles[ls_index]

G_dict = {"New algo": G_new, "trackmate: nearest neighbor": G_NN, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}
G_list = sorted(G_dict)
g_index = 1
G_current = G_dict[G_list[g_index]]


image = composer.get_single_frame_phase(frame)
ax = visualizer.subplot_single_frame_phase(ax=ax, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
disconnect_zoom = zoom_factory(ax)

# Update plot function
def update_plot(frame, ax, fig):
    global label_styles, labels, li_index, ls_index, G_dict, G_list, g_index

    label_info = labels[li_index]
    label_style = label_styles[ls_index]

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    G_current = G_dict[G_list[g_index]]
    
    ax.clear()
    image = composer.get_single_frame_phase(frame)
    ax =  visualizer.subplot_single_frame_phase(ax=ax, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    disconnect_zoom = zoom_factory(ax)
    fig.suptitle(f"Frame: {frame}, use keyboard: ⬅️ ➡️ to change frame, use 'c' to circle label, use 1 to change cell tag \n Algo: {G_list[g_index]}", color = "blue")
    fig.canvas.draw_idle()
    return ax

def on_key(event):
    global frame, ax, max_frame, fig, li_index , ls_index
    if event.key in ['right', 'down']:
        frame = min(frame + 1, max_frame) 
        update_plot(frame, ax, fig)
    elif event.key in ['left', 'up']:
        frame  = max(frame - 1, 0)
        update_plot(frame, ax, fig)
    elif event.key == 'c':
        ls_index += 1
        ls_index =  ls_index % len(label_styles)
        update_plot(frame, ax, fig)
    elif event.key == '1':
        li_index += 1
        li_index =  li_index % len(labels)
        update_plot(frame, ax, fig)
    elif event.key in ['g', 'G']:
        g_index  += 1
        g_index  =  g_index  % len(G_list)
        update_plot(frame, ax, fig)
    

        

fig.suptitle(f"Frame: {frame}, use keyboard: ⬅️ ➡️, use 'l' to change label, use c to change color/tag", color = "blue")
fig.canvas.mpl_connect('key_press_event', lambda event: on_key(event))

# Show the plot in a separate window
plt.show()

"""


'\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport ipywidgets as widgets\nfrom mpl_interactions import zoom_factory\nfrom IPython.display import display\nimport matplotlib\n\nplt.close(\'all\')\nmatplotlib.use(\'TkAgg\')  # Make sure this backend is compatible with your environment\n\nframe = 0  # Initialize frame\nmax_frame = composer.frame_num - 1\n\n# Assuming visualizer, composer, and G are defined\n# Create the initial plot\nplt.ioff()\nfig, ax = plt.subplots(figsize=(10, 10))\nfig.tight_layout()\n\n\n\nlabel_info_1 = visualizer.get_label_info(G)\nlabel_info_2 = visualizer.get_generation_label_info(G)\n\n\nlabels = [label_info_1, label_info_2]\nli_index = 0\nlabel_info = labels[li_index]\n\n\nlabel_styles = ["regular", "circled", "empty"]\nls_index = 0\nlabel_style = label_styles[ls_index]\n\nG_dict = {"New algo": G_new, "trackmate: nearest neighbor": G_NN, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}\nG_list = sorted(G_dict)\ng_index = 1\nG_current = G_dict[

In [18]:
"""
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from mpl_interactions import zoom_factory
from IPython.display import display
import matplotlib

plt.close('all')
matplotlib.use('TkAgg')  # Make sure this backend is compatible with your environment

frame = 0  # Initialize frame
max_frame = composer.frame_num - 1

# Assuming visualizer, composer, and G are defined
# Create the initial plot
plt.ioff()
fig, ax = plt.subplots(figsize=(10, 10))
fig.tight_layout()

G_dict = {"New algo": G_new, "trackmate: nearest neighbor": G_NN, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}
G_list = sorted(G_dict)
g_index = 0
G_current = G_dict[G_list[g_index]]


labels = {}
for name, G in G_dict.items():
    label_info_1 = visualizer.get_label_info(G)
    label_info_2 = visualizer.get_generation_label_info(G)
    labels[name] = [label_info_1, label_info_2]

li_index = 0
label_info = labels[G_list[g_index]][li_index]


label_styles = ["regular", "circled", "empty"]
ls_index = 0
label_style = label_styles[ls_index]


image = composer.get_single_frame_phase(frame)
ax = visualizer.subplot_single_frame_phase(ax=ax, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
disconnect_zoom = zoom_factory(ax)

# Update plot function
def update_plot(frame, ax, fig):
    global label_styles, labels, li_index, ls_index, G_dict, G_list, g_index

    label_info = labels[G_list[g_index]][li_index]
    label_style = label_styles[ls_index]

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    G_current = G_dict[G_list[g_index]]
    
    ax.clear()
    image = composer.get_single_frame_phase(frame)
    ax =  visualizer.subplot_single_frame_phase(ax=ax, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    disconnect_zoom = zoom_factory(ax)
    fig.suptitle(f"Frame: {frame}.  Algorithmn: {G_list[g_index]}", color = "blue")
    fig.canvas.draw_idle()
    return ax

def on_key(event):
    global frame, ax, max_frame, fig, li_index , ls_index, g_index
    if event.key in ['right', 'down']:
        frame = min(frame + 1, max_frame) 
        update_plot(frame, ax, fig)
    elif event.key in ['left', 'up']:
        frame  = max(frame - 1, 0)
        update_plot(frame, ax, fig)
    elif event.key == 'c':
        ls_index += 1
        ls_index =  ls_index % len(label_styles)
        update_plot(frame, ax, fig)
    elif event.key == '1':
        li_index += 1
        li_index =  li_index % len(labels)
        update_plot(frame, ax, fig)
    elif event.key in ['g', 'G']:
        g_index  += 1
        g_index  =  g_index  % len(G_list)
        update_plot(frame, ax, fig)
    

        

fig.suptitle(f"Frame: {frame}, use keyboard: ⬅️ ➡️, use 'l' to change label, use c to change color/tag", color = "blue")
fig.canvas.mpl_connect('key_press_event', lambda event: on_key(event))

# Show the plot in a separate window
plt.show()

"""

'\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport ipywidgets as widgets\nfrom mpl_interactions import zoom_factory\nfrom IPython.display import display\nimport matplotlib\n\nplt.close(\'all\')\nmatplotlib.use(\'TkAgg\')  # Make sure this backend is compatible with your environment\n\nframe = 0  # Initialize frame\nmax_frame = composer.frame_num - 1\n\n# Assuming visualizer, composer, and G are defined\n# Create the initial plot\nplt.ioff()\nfig, ax = plt.subplots(figsize=(10, 10))\nfig.tight_layout()\n\nG_dict = {"New algo": G_new, "trackmate: nearest neighbor": G_NN, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}\nG_list = sorted(G_dict)\ng_index = 0\nG_current = G_dict[G_list[g_index]]\n\n\nlabels = {}\nfor name, G in G_dict.items():\n    label_info_1 = visualizer.get_label_info(G)\n    label_info_2 = visualizer.get_generation_label_info(G)\n    labels[name] = [label_info_1, label_info_2]\n\nli_index = 0\nlabel_info = labels[G_list[g_index]][li_index]\n\n\nlabel_

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from mpl_interactions import zoom_factory
from IPython.display import display
import matplotlib

from visualizer import CELL_EVENT_COLOR

plt.close('all')
matplotlib.use('TkAgg')  # Make sure this backend is compatible with your environment

frame = 0  # Initialize frame
max_frame = composer.frame_num - 1

# Assuming visualizer, composer, and G are defined
# Create the initial plot
plt.ioff()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [5, 2]})
fig.tight_layout()


G_dict = {"New algo": G_new, "trackmate: nearest neighbor": G_NN, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}
G_list = sorted(G_dict)
g_index = 0
G_current = G_dict[G_list[g_index]]


labels = {}
tags = {}
poss = {}

for name, G in G_dict.items():
    label_info_1 = visualizer.get_label_info(G)
    label_info_2 = visualizer.get_generation_label_info(G)
    labels[name] = [label_info_1, label_info_2]
    tags[name] = visualizer.tag_type(G)
    poss[name] = visualizer.get_lineage_pos(G)


li_index = 0
label_info = labels[G_list[g_index]][li_index]
tag = tags[G_list[g_index]]
pos = poss[G_list[g_index]]


label_styles = ["regular", "circled", "empty"]
ls_index = 0
label_style = label_styles[ls_index]

cells_s = {CELL_EVENT_COLOR[CellEvent.BIRTH]: tag[CellEvent.BIRTH], CELL_EVENT_COLOR[CellEvent.DIE]: tag[CellEvent.DIE]}
edges_s = {CELL_EVENT_COLOR[CellEvent.SPLIT]: tag[CellEvent.SPLIT], CELL_EVENT_COLOR[CellEvent.MERGE]: tag[CellEvent.MERGE]}
        

image = composer.get_single_frame_phase(frame)
ax1 = visualizer.subplot_single_frame_phase(ax=ax1, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
ax1.set_title(f"Frame: {frame}. Algo:{G_list[g_index]}", weight = 600) 

ax2 = visualizer.subplot_lineage(ax2, G_current, pos, with_background=True, nodes_special=cells_s, edges_special=edges_s)
ax2.set_axis_off()
disconnect_zoom = zoom_factory(ax1)

# Update plot function
def update_plot(frame, ax1, ax2, fig, update_ax2=False):
    global label_styles, labels, li_index, ls_index, G_dict, G_list, g_index

    label_info = labels[G_list[g_index]][li_index]
    label_style = label_styles[ls_index]
    

    xlim = ax1.get_xlim()
    ylim = ax1.get_ylim()

    G_current = G_dict[G_list[g_index]]
    
    ax1.clear()
    image = composer.get_single_frame_phase(frame)
    ax1 = visualizer.subplot_single_frame_phase(ax=ax1, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)
    ax1.set_title(f"Frame: {frame}. Algo:{G_list[g_index]}", weight = 600) 

    disconnect_zoom = zoom_factory(ax1)
    fig.canvas.draw_idle()

    if update_ax2 :
        ax2.clear()
        tag = tags[G_list[g_index]]
        pos = poss[G_list[g_index]]
        cells_s = {CELL_EVENT_COLOR[CellEvent.BIRTH]: tag[CellEvent.BIRTH], CELL_EVENT_COLOR[CellEvent.DIE]: tag[CellEvent.DIE]}
        edges_s = {CELL_EVENT_COLOR[CellEvent.SPLIT]: tag[CellEvent.SPLIT], CELL_EVENT_COLOR[CellEvent.MERGE]: tag[CellEvent.MERGE]}
        ax2 =  visualizer.subplot_lineage(ax2, G_current, pos, with_background=True, nodes_special=cells_s, edges_special=edges_s)
        ax2.set_axis_off()

    return ax1, ax2


def on_key(event):
    global frame, ax1, ax2, max_frame, fig, li_index , ls_index, g_index
    if event.key in ['right', 'down']:
        frame = min(frame + 1, max_frame) 
        update_plot(frame, ax1, ax2,  fig)
    elif event.key in ['left', 'up']:
        frame  = max(frame - 1, 0)
        update_plot(frame, ax1, ax2,  fig)
    elif event.key == 'c':
        ls_index += 1
        ls_index =  ls_index % len(label_styles)
        update_plot(frame, ax1, ax2,  fig)
    elif event.key == '1':
        li_index += 1
        li_index =  li_index % len(labels[G_list[0]])
        update_plot(frame, ax1, ax2, fig)
    elif event.key in ['g', 'G']:
        g_index  += 1
        g_index  =  g_index  % len(G_list)
        update_plot(frame, ax1, ax2, fig, update_ax2= True)
    

        

fig.canvas.mpl_connect('key_press_event', lambda event: on_key(event))

# Show the plot in a separate window
plt.show()
